### Capturando dados

In [ ]:
# Carregamento de dados
import pandas as pd
df= pd.read_csv('../dados/data_polarity.csv', low_memory=False, index_col=[0])

print(";---------------------;")
print("|  Dados Carregados!  |")
print("|---------------------|")
print("|  df: %d         |" %len(df))
print(";---------------------;")

In [ ]:
# Análise de Sentimentos com custom_vader
import sys
if '../' not in sys.path: sys.path.append('../')
from models.custom_vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer("../dados/custom_vader_lexicon.txt")
print(sia.polarity_scores('eu gostaria ir ao mercado comprar pão'))

In [ ]:
# criando funçoes
import numpy as np
import re
import nltk
import html
import unidecode

def remove_stopwords(text):
    stopwords = list()
    try: stopwords = nltk.corpus.stopwords.words("portuguese")
    except: nltk.download('stopwords')
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords += ['oi', 'ola', 'opa', 'oie', 'oii', 'oiee', 'oin', 'só', 'so']
    stopwords = [w for w in stopwords if w not in ['mas', 'muito']]
    reg_caracters='\!|\"|\#|\$|\%|\&|\|\'|\(|\)|\*|\+|\,|\.|\/|\n|\:|\;|\<|\=|\>|\?|\@|\[|\\|\]|\^|\_|\`|\{\|\}|\~|[0-9]*'
    remove_acents = lambda sentence: ' '.join([unidecode.unidecode(s) if s.isalnum() else s for s in sentence.split()])
    
    # remove link do texto
    text = re.sub(r'http\S+', '', text) 
    # adiciona espaço no inicio e final do hex-emoji
    text = text.replace("&#x", " &#x").replace(';', '; ') 
    # transforma o hex-emoji em emoji
    text=html.unescape(text)
    # remove caracteres especiais do texto
    text = re.sub(reg_caracters, "", text)
    # remove acentos das palavras preservando 
    text = remove_acents(text)

    return " ".join([s for s in text.split() if s not in stopwords])

condictions = lambda pol: [pol < -.45, -.45 <= pol < -.2, -.2 <= pol < .2, .2 <= pol < .45, pol > .45]
describes_emojis = {'negativo': "😡", 'irrazoavel': "🙁", 'neutro': "😐", 'razoavel': "🙂", 'positivo': "🤩"}


get_polarity = lambda sentence: sia.polarity_scores(sentence, prin_not_lex=False)[0]["compound"]
get_sentence_size = lambda sentence: len(remove_stopwords(sentence).split())

#### Análise das primeiras 1k msgs

In [ ]:
# analisando msgs

test = df[df['from_me']==0].head(1000).copy()
test = test[['protocolo', 'log_texto', 'from_me', 'emojis']]
test['polarity'] = test['log_texto'].apply(get_polarity)
test['sentence_size'] = test['log_texto'].apply(get_sentence_size)
test['class'] = test["polarity"].apply(lambda p: np.select(condictions(p), [*describes_emojis.keys()]) if p is not None else 'indefinido')


In [ ]:
sia.polarity_scores(remove_stopwords(test.log_texto.iat[0]))
remove_stopwords(test.log_texto.iat[0])

In [ ]:
test["class"].value_counts()
test[test["class"] == "negativo"]

In [ ]:
# Graphs
import plotly.graph_objects as go
from models.graph_tools import Graph

# fig=Graph(test, "Análise de Polaridade por qtd. de Palavras", is_subplot=True, cols=2, cols_width=[.15,.85], rows=1, row_width=[1])
fig=Graph(test, "Análise de Polaridade por qtd. de Palavras")
fig.line('sentence_size', ['polarity'], modes=['markers'], 
    marker_colorscale='magenta', 
    hovertemplate='p:%{y:,.2f}%<br>w:%{x}'
)
fig.show().update_layout(
    paper_bgcolor='rgba(230,230,230,230)', 
    # yaxis={'title': "Polaridade", 'range': [-1, 1]},
    xaxis={"title":"Qnd.Palavras na Frase"},
    title_x=.5
)

# fig.show().update_layout(yaxis_showticklabels=True, yaxis2_showticklabels=False)

In [ ]:

fig=Graph(test, "Análise de Polaridade por qtd. de Palavras")

fig.show().update_layout(
    paper_bgcolor='rgba(230,230,230,230)', 
    # yaxis={'title': "Polaridade", 'range': [-1, 1]},
    xaxis={"title":"Qnd.Palavras na Frase"},
    title_x=.5
)
fig.show().add_trace(
    go.Box(
        y=test.query('polarity > .2')['sentence_size'], 
        marker_color="seagreen", boxpoints=False, 
        name='positivos', jitter=0.5, pointpos=-0, boxmean=True
    )
)

fig.show().add_trace(
    go.Box(
        y=test[test.polarity.between(-.2, .2)]['sentence_size'], 
        # y=test.query('polarity < -.2')['sentence_size'], 
        marker_color="steelblue", boxpoints=False, 
        name='neutros', jitter=.5, pointpos=-0, boxmean=True
    )
)
fig.show().add_trace(
    go.Box(
        y=test.query('polarity < -.2')['sentence_size'], 
        marker_color="#FF851B", boxpoints=False, 
        name='negativos', jitter=.5, pointpos=-0, boxmean=True
    )
)




### Treinamento

In [ ]:
# Análise de Sentimentos com custom_vader
import sys
if '../' not in sys.path: sys.path.append('../')
from models.custom_vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer("../dados/custom_vader_lexicon.txt", 
  stopwords_file = "/home/dba/Documentos/nlp-api/dados/custom_stopwords.txt")

sentences = [
  {"msg_id": 1, "msg": 'Me orientaram a entrar em contato com vocês.'},
  {"msg_id": 2, "msg": 'Adorei o atendimento daquele rapazinho'},
  {"msg_id": 3, "msg": 'Na verdade vou fazer o cancelamento do meu plano'},
  {"msg_id": 4, "msg": 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam magna pretium vehicula.'}
]

res = [
  {"msg_id": 1, "polarity": 0, "describe": "neutro", "status": True},
  {"msg_id": 2, "polarity": 0.875, "describe": "positivo", "status": True},
  {"msg_id": 3, "polarity": -0.3818, "describe": "irrazoavel", "status": True},
  {"msg_id": 4, "describe": "indefinido", "status": False}
]



In [ ]:
from fastapi import FastAPI, Request
from routes.unit_polarity import SucessResponse, ValidationErrorResponse, ServerErrorResponse, get

# Análise de Sentimentos com custom_vader
condictions = lambda pol: [pol is None, pol < -.45, -.45 <= pol < -.2, -.2 <= pol < .2, .2 <= pol < .45, pol > .45]
describes_emojis = {"indefinido": "😵‍💫", 'negativo': "😡", 'irrazoavel': "🙁", 'neutro': "😐", 'razoavel': "🙂", 'positivo': "🤩"}

In [ ]:
import json
import pandas as pd

from fastapi.responses import JSONResponse


def __loop_searches(sentences:list) -> list:
    """ ## obtem a polaridade para cada sentença
    @sentences: lista de sentenças;
    """

    result = []
    for sentence in sentences:
        response = json.loads(get(Request, sentence['msg'], sia, condictions, describes_emojis).body)
        response['msg_id'] = sentence['msg_id']
        result.append(response)

    return result

def __concat_sentences(sentences: list, results: list) -> list:
    """ 
    ## obtem a polaridade de todas sentenças juntas
    ### e retorna uma string concatenada
    @sentences: lista de sentenças;
    @results: lista de resultados da obtenção da polaridade das sentenças
    """

    df = pd.DataFrame(sentences).set_index('msg_id').join(pd.DataFrame(results).set_index('msg_id'))
    conversation = ' '.join([s.msg for i, s in df.iterrows() if s.sucess])
    return conversation
    
def multi_polarity(request: Request, sentences: list, sia, condictions: list, describes_emojis: list):
    """ 
    ## Consulta Multipla de Polaridade de Sentença
    request example: \n
    ```
    sentences = [
        {"msg_id": 1, "msg": 'Me orientaram a entrar em contato com vocês.'},
        {"msg_id": 4, "msg": 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam magna pretium vehicula.'}
    ]

    multi_polarity(sentences)

    >> [
         {'event': 'unit_polarity','sucess': True,'polarity': 0.3182, 'describe': 'razoavel', 'describe_emoji': '🙂', 'msg_id': 1},
         {'event': 'unit_polarity','sucess': False, 'describe': 'indefinido', 'describe_emoji': '😵', 'msg_id': 4},
         {'event': 'multi_polarity','sucess': False, 'describe': 'indefinido', 'describe_emoji': '😵', 'msg_id': 4},
       ]
    ```
    """

    try: 
        sentences_result = __loop_searches(sentences)
        sentence = __concat_sentences(sentences, sentences_result)
        result = get(Request, sentence, sia, condictions, describes_emojis)

        response_body = json.loads(result.body)
        response_body['event'] = "multi_polarity"
        response_body['sentences'] = sentences_result
        response_body = json.dumps(response_body)

        result.body = str(response_body)

        return result

    except Exception as e: 
        response = {"event": "multi_polarity", "sucess": False}
        response["describe"] = str(e)
        # __log(sentence, None, e, request)
        return JSONResponse(status_code=500, content=response)


a=multi_polarity(Request, sentences, sia, condictions, describes_emojis)
json.loads(a.body)

In [ ]:
sia.polarity_scores("Eu não gostei do hotel. A cama era ruim.")

In [ ]:
sentences = [
    {"sentence": 'Me orientaram a entrar em contato com vocês.'},
    {"sentence": 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam magna pretium vehicula.'}
]

a= {
    "event": "unit_polarity",
    "sucess": False,
    "polarity": 0.3182,
    "describe": "razoavel",
    "describe_emoji": "�",
}

df = pd.DataFrame(sentences).join(pd.DataFrame([a]))
df

In [ ]:
a= {
    "event": "unit_polarity",
    "sucess": False,
    "polarity": 0.3182,
    "describe": "razoavel",
    "msg_id": 1
}
pd.DataFrame([a])["sucessw"]

In [ ]:
sentences=[
  {"sentence": "Me orientaram a entrar em contato com vocês."},
  {"sentence": "Adorei o atendimento daquele rapazinho"},
  {"sentence": "Na verdade vou fazer o cancelamento do meu plano"},
  {"sentence": "Na verdade vou fazer o cancelamento do meu plano"},
]

not all("sentence" in s.keys() for s in sentences)

type(1) is not list

In [ ]:
import http.client
import json

connection = http.client.HTTPSConnection('192.168.0.41', 8560, timeout=10)
headers = {'Content-type': 'application/json'}

foo = {'text': 'Hello world github/linguist#1 **cool**, and #1!'}
json_foo = json.dumps(foo)

connection.request('GET', '/polarity/unique/?sentence', json_foo, headers, verify=False)

response = connection.getresponse()
print(response.read().decode())

### Testes Unitários

In [ ]:
import requests
import json
headers = {'Content-type': 'application/json'}

sentences= [
  {"sentence": "Me orientaram a entrar em contato com vocês."},
  {"sentence": "Adorei o atendimento daquele rapazinho"},
  {"sentencae": "Na verdade vou fazer o cancelamento do meu plano"},
]

all_invalid_keys= None

sentence = "Adorei o atendimento daquele rapazinho"


body = str(all_invalid_keys).replace("'", '"').encode()

result = requests.get(f'http://192.168.0.41:8560/v1/polarity/unique/?sentence={sentence}')
# result = requests.post('http://192.168.0.41:8560/v1/polarity/list' , body, headers=headers)

print(result.status_code)

print(json.loads(result.content))


In [ ]:
result = requests.get(f'http://192.168.0.41:8560/v1/polarity/unique/?sentence={sentence}')
result

In [ ]:
import grequests

url = "http://192.168.0.41:8560/v1/polarity/unique"

valids_sentences= [
    "1 Me orientaram a entrar em contato com vocês.",
    "2 Adorei o atendimento daquele rapazinho",
    "3 Na verdade vou fazer o cancelamento do meu plano"
]

invalids_sentences = ["4", '', 'None']

valids_urls = [f'{url}/?sentence={s}' for s in valids_sentences] * 3
invalids_urls = [f'{url}/?sentence={s}' for s in invalids_sentences] * 3

rs = (grequests.get(u, gtimeout=3) for u in invalids_urls)
results = grequests.map(rs)

In [ ]:
ip_host="192.168.0.41"
port_host=8560
route= "v1/polarity/unique"
route_list= "v1/polarity/list"
url = f'http://{ip_host}:{port_host}/{route_list if route_list[0] != "/" else route_list[1:]}'
headers = {'Content-type': 'application/json'}


# MODEL
multi_valid_sentences= [
    {"sentence": "Me orientaram a entrar em contato com vocês."},
    {"sentence": "Adorei o atendimento daquele rapazinho"},
    {"sentence": "Na verdade vou fazer o cancelamento do meu plano"},
] 

multi_invalid_sentences= [
    {"sentence": "11111111111111"},
    {"sentence": "None"},
    {"sentence": ""},
] 

multi_invalid_keys= [
    {"wrog_key1": "Me orientaram a entrar em contato com vocês."},
    {"wrog_key2": "Adorei o atendimento daquele rapazinho"},
    {"wrog_key3": "Na verdade vou fazer o cancelamento do meu plano"},
] 

results = []
for i, body in enumerate([multi_valid_sentences, multi_invalid_sentences, multi_invalid_keys]):
    body = str(body).replace("'", '"').encode()

    rs = (grequests.post(link, data=body, headers=headers) for link in [url]*500) 
    results = grequests.map(rs)

    assert not any([r is None for r in results]), "Some request failed"

    if i == 0: assert all([r.status_code == 200 for r in results]), "All state codes should be 200"
    if i in [1,2]: assert all([400 <= r.status_code <= 406 for r in results]), "All state codes should be between 400~499"

df=pd.DataFrame({'reqs':[r.elapsed.total_seconds()  if r is not None else None for r in results], 'a': [1]*len(results)})
df

In [ ]:
import pandas as pd
df=pd.DataFrame({'reqs':[r.elapsed.total_seconds()  if r is not None else None for r in results], 'a': [1]*len(results)})
df.reqs.max()
df


In [ ]:
import plotly.express as px

fig = px.line(df.reset_index(), x="index", y="reqs", title='Time Series with Rangeslider')
fig.show()